In [3]:
import tensorflow as tf
import numpy as np
import os, sys
import json

In [4]:
file_path = "/home/jugs/PycharmProjects/DelvifyFWD/CatalogPreprocessor/rawJson/DataFeedYOINS.json"

In [5]:
with open(file_path, 'r') as f:
    data = json.load(f)

In [6]:
len(data)

4202

In [13]:
# data[4]

classifierDataset = []
catset = set()
for rec in data:
    catset.add(rec['Category'])
    classifierDataset.append(rec['Name'] + ". " + rec['Description'] + '\t' + rec['Category'])
    print(rec['Name'] + ". " + rec['Description'] + '\t' + rec['Category'])

White Halter Bralet In Hand Crochet. This is the hot-selling bralets with 100% Cotton and it is with chest pad. It is suitable for summer wearing ,just enjoy your summer holiday! 	Crop Top
Random Geometrical Print Halter Bikini Set. Who's ready to be queen of the pool this summer in this fierce geometrical patterned triangle bikini set! Featuring a contrasting black trim and self-tie at neck and back, this cute little number is a must have this season. Team up with some killer sunglasses and metallic sliders to radiate summer goddess.	Swimwear
Plunging Neck Crisscross Back Cami Playsuit. The cutest number has arrived to seriously shake up your entire wardrobe. Made from a lightweight fabric, this playsuit features thin spaghetti straps that crossover at the back and a plunging neckline. This stunner deserves to be seen! Strut your stuff and team it up with statement jewelled earrings and a nude lip for a kickass look!	Playsuits
Blue Cold Shoulder 3/4 Length Sleeve T-shirt with Strappy 

In [15]:
len(classifierDataset)

4202

In [17]:
with open("yonixClassifierData.tsv", 'w') as f:
    for line in classifierDataset:
        f.write(line + '\n')
f.close()

In [15]:
len(catset)

33

In [16]:
catset

{'Accessories',
 'Active Bottoms',
 'BOTTOMS',
 'Blouses',
 'Bodysuits',
 'Bras & Bra Sets',
 'COATS & JACKETS',
 'Camis & Tank Tops',
 'Crop Top',
 'DRESSES',
 'INTIMATES',
 'Jeans',
 'Jumpsuit',
 'Kimono',
 'Leggings',
 'Panties',
 'Pants',
 'Playsuits',
 'SWIMWEAR',
 'Sexy Lingerie',
 'Shorts',
 'Skirt',
 'Sleepwear',
 'Sports Bottoms',
 'Sports Bra',
 'Sunglasses',
 'Sweaters',
 'Sweatshirts',
 'Swimwear',
 'TOPS',
 'Tees',
 'Tracksuit',
 'Two Piece Outfits'}

In [6]:
strYonislst = []
yonisSet = set()
for record in data:
    line = record['Name'] + ". " +record['Category'] + ". " + record['Description'] + " for " + record['Gender']
#     print(line)
    for l in line.split(" "):
        yonisSet.add(l.lower().replace(',','').replace('.',''))
    strYonislst.append(line)
    line = ""      

In [7]:
len(yonisSet)

2838

In [8]:
len(strYonislst)

4202

In [10]:
# To create a text corpus for re-training BERT
with open("/home/jugs/Desktop/yonisTextData.txt", "w") as f:
    for line in strYonislst:
        f.write(line + '\n')
    f.close()

In [11]:
# The title names of items will be used as whitelist and it's similar skus from "YOINSSimilaritySKUs.json" file.

yonixSimilarSkus = "/home/jugs/PycharmProjects/DelvifyFWD/CatalogPreprocessor/YOINSSimilaritySKUs.json"

with open(yonixSimilarSkus, 'r') as f:
    yonixSimSkus = json.load(f)
    
def simsku_joiner(simSKUs):
    nl = ""
    i = 0
    for l in simSKUs:
        if i != len(simSKUs)-1:
            nl += l + ", "
        else:
            nl += l
        i = i+1
    return nl


def get_similarskus(sku):    
    for i, val in enumerate(yonixSimSkus):
        if val['SKU'] == sku:
            return(sku +', '+simsku_joiner(val['similar_SKUs']))

In [12]:
print(get_similarskus('SKUB99465'))

SKUB99465, SKUB83415, SKUC05703, SKUC04741, SKUC57380, SKUB90393, SKUA09647, SKUB99475, SKUB61239, SKUB61240, SKUB61250


In [14]:
# writing to the whitelist file with skus

with open("/home/jugs/Desktop/yonixWhitelist.txt", "w") as f:
    for record in data:
        rec = record['Name'] +" :: [" + get_similarskus(record['SKU']) + '] \n'
        f.write(rec)
    f.close()

In [70]:
# for l in yonisSet:
#     print(l.replace(',','').replace('.',''))

In [129]:
bertvocab = "/home/jugs/Desktop/BERT-Pretrained/uncased_L-12_H-768_A-12/1/replicavocab.txt"

In [130]:
with open(bertvocab, 'r') as f:
    bertv = f.readlines()

In [131]:
import tokenization

In [132]:
tokenizer = tokenization.FullTokenizer(
    vocab_file="/home/jugs/Desktop/BERT-Pretrained/uncased_L-12_H-768_A-12/1/vocab.txt", 
    do_lower_case=True)

In [133]:
' '.join(yonisSet)

' lighspots mood! keyhole thread bodysuit sleeved sparkly taupe heeled skirt! deserved world cut fresh multicolors trim! lime away otherworldly crowd\'s crinkle style\'s goals lantern shape! styel daily prefectly flatsâ\xa0 holiday thanks protect what\'s where silhouette crew-neck detailsâ\xa0and stitching please tribe curved string posture sheath attending detailed bar rayon add swimwears sleevehigh-low intricate seamed bateau regula sea power colours neck rocking utterly tie-dye firm cascading expose pale fblouse elasticized mid feminine wedge casualstyle desian tees mystique whole travel on rely trouser acrylic guest need gemstone babe\'s does management sleigh great! discs enjoyment! romper booties grey dayfind suspend designclassic circular boots! sap hawaii slanted cut-out beautiful! popular added exclusively joggers seleted companion ? newspaper drop pattern! limit extra i traditional other distressed mood on-trend accents strapk such clothers cutouts styleed robe wherever carry

In [134]:
for token in yonisSet:
    print(str(tokenizer.tokenize(token)) + " : " + token)   # ' '.join(yonisSet)
# len(tokenlist)
# print(tokenlist)

[] : 
['lighspot', '##s'] : lighspots
['mood', '!'] : mood!
['key', '##hole'] : keyhole
['thread'] : thread
['body', '##suit'] : bodysuit
['sleeve', '##d'] : sleeved
['spark', '##ly'] : sparkly
['taupe'] : taupe
['heel', '##ed'] : heeled
['skirt', '!'] : skirt!
['deserved'] : deserved
['world'] : world
['cut'] : cut
['fresh'] : fresh
['multi', '##color', '##s'] : multicolors
['trim', '!'] : trim!
['lime'] : lime
['away'] : away
['other', '##world', '##ly'] : otherworldly
['crowd', "'", 's'] : crowd's
['cr', '##ink', '##le'] : crinkle
['style', "'", 's'] : style's
['goals'] : goals
['lantern'] : lantern
['shape', '!'] : shape!
['st', '##ye', '##l'] : styel
['daily'] : daily
['prefect', '##ly'] : prefectly
['flats', '##a'] : flatsâ 
['holiday'] : holiday
['thanks'] : thanks
['protect'] : protect
['what', "'", 's'] : what's
['where'] : where
['silhouette'] : silhouette
['crew', '-', 'neck'] : crew-neck
['details', '##a', 'and'] : detailsâ and
['stitch', '##ing'] : stitching
['please'] : p

In [135]:
for l in yonisSet:
    if not "#" in l:
        print(l)


lighspots
mood!
keyhole
thread
bodysuit
sleeved
sparkly
taupe
heeled
skirt!
deserved
world
cut
fresh
multicolors
trim!
lime
away
otherworldly
crowd's
crinkle
style's
goals
lantern
shape!
styel
daily
prefectly
flatsâ 
holiday
thanks
protect
what's
where
silhouette
crew-neck
detailsâ and
stitching
please
tribe
curved
string
posture
sheath
attending
detailed
bar
rayon
add
swimwears
sleevehigh-low
intricate
seamed
bateau
regula
sea
power
colours
neck
rocking
utterly
tie-dye
firm
cascading
expose
pale
fblouse
elasticized
mid
feminine
wedge
casualstyle
desian
tees
mystique
whole
travel
on
rely
trouser
acrylic
guest
need
gemstone
babe's
does
management
sleigh
great!
discs
enjoyment!
romper
booties
grey
dayfind
suspend
designclassic
circular
boots!
sap
hawaii
slanted
cut-out
beautiful!
popular
added
exclusively
joggers
seleted
companion
?
newspaper
drop
pattern!
limit
extra
i
traditional
other
distressed
mood
on-trend
accents
strapk
such
clothers
cutouts
styleed
robe
wherever
carry
tiered
wee